In [3]:
# 답변 파일 생성(sample_submission.csv)
# llm 변경후 사용.

In [4]:
# # 1. 데이터 파일 읽기
# import pandas as pd
# from dotenv import load_dotenv

# load_dotenv()

# df = pd.read_csv("./data/summary_one.csv")
# df.tail()


# 1. 데이터 파일 읽기 (documents.jsonl)
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

# .jsonl 파일을 읽기 위해 read_json 사용 (lines=True 옵션 필수)
df = pd.read_json("../korea202/data/documents.jsonl", lines=True)

# 기존 코드와의 호환성을 위해 content를 summary 열로 복사
# 벡터 DB 생성 시 'summary' 열을 사용하기 때문입니다.
df['summary'] = df['content']

df.tail()

,docid,src,content,summary
4267,ae28101b-a42e-45b7-b24b-4ea0f1fb2d50,ko_ai2_arc__ARC_Challenge__train,비뇨기계와 순환계는 혈액이 신장을 통과하면서 폐기물과 물이 제거될 때 관여하는 두 ...,비뇨기계와 순환계는 혈액이 신장을 통과하면서 폐기물과 물이 제거될 때 관여하는 두 ...
4268,eb727a4f-29c7-4d0c-b364-0e67de1776e9,ko_ai2_arc__ARC_Challenge__train,로봇은 현대 산업에서 많은 역할을 수행할 수 있습니다. 그러나 로봇 사용의 중대한 ...,로봇은 현대 산업에서 많은 역할을 수행할 수 있습니다. 그러나 로봇 사용의 중대한 ...
4269,0c8c0086-c377-4201-81fa-25159e5435a7,ko_mmlu__human_sexuality__test,"월경은 여성의 생리주기에 따라 발생하는 현상으로, 에스트로겐과 프로게스테론 수치의 ...","월경은 여성의 생리주기에 따라 발생하는 현상으로, 에스트로겐과 프로게스테론 수치의 ..."
4270,06da6a19-ec78-404e-9640-9fc33f63c6a2,ko_ai2_arc__ARC_Challenge__train,식물이 내뿜는 가스는 산소입니다. 식물은 광합성 과정을 통해 태양 에너지를 이용하여...,식물이 내뿜는 가스는 산소입니다. 식물은 광합성 과정을 통해 태양 에너지를 이용하여...
4271,03c36d5e-c711-4dc2-b4db-aaeb94d86395,ko_mmlu__computer_security__test,"버퍼 오버런은 개발자들이 만든 널리 퍼져 있는 앱의 코딩 오류로써, 공격자가 시스템...","버퍼 오버런은 개발자들이 만든 널리 퍼져 있는 앱의 코딩 오류로써, 공격자가 시스템..."


In [5]:
# 2. 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4272 entries, 0 to 4271
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   docid    4272 non-null   object
 1   src      4272 non-null   object
 2   content  4272 non-null   object
 3   summary  4272 non-null   object
dtypes: object(4)
memory usage: 133.6+ KB


In [6]:
# 3. 디비 생성 클래스

import faiss
import torch
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore

def create_empty_faiss(embeddings, use_cosine=True):
    """빈 FAISS 벡터스토어 생성"""
    dimension = len(embeddings.embed_query("test"))
    print(f"dimension={dimension}")
    if use_cosine:
        index = faiss.IndexFlatIP(dimension)
        normalize = True
    else:
        index = faiss.IndexFlatL2(dimension)
        normalize = False
    
    return FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=normalize
    )

def get_embeddinggemma_300m(opt):
    return HuggingFaceEmbeddings(
        model_name="google/embeddinggemma-300m",
        encode_kwargs={"prompt_name": opt}
    )

def get_qwen3_embedding_4b():
    return HuggingFaceEmbeddings(
        model_name="Qwen/Qwen3-Embedding-4B",
        model_kwargs={
            "device": "cuda"
        },
        encode_kwargs={
            "normalize_embeddings": True
        }
    )

class ScienceRAG:
    def __init__(self):
        self.embeddings = HuggingFaceEmbeddings(
                                    model_name="BAAI/bge-m3",
                                    #model_name="Qwen/Qwen3-Embedding-0.6B",
                                    model_kwargs={"device": "cuda"} ,
                                    encode_kwargs={"normalize_embeddings": True}) 
        
        #self.embeddings = get_qwen3_embedding_4b()
        
        self.vectorstore = create_empty_faiss(self.embeddings, use_cosine=False) 
        
    
    def add_documents(self, df):
        """문서를 요약하여 벡터DB에 저장"""
        
        texts = df['summary'].tolist()
        metadatas = [
            {
                'docid': row['docid'],
                'content': row['content']
            } 
            for _, row in df.iterrows()
        ]
        ids = df['docid'].tolist()


        # 2. 검색용 요약을 벡터DB에 저장
        self.vectorstore.add_texts(
            texts,
            metadatas=metadatas,
            ids = ids
        )
        
       
    def search(self, query: str, k: int = 3):
        """요약된 내용으로 검색"""
        results = self.vectorstore.similarity_search(query, k=k)
        return results

In [7]:
# 4. 디비 생성 

science_rag = ScienceRAG()

science_rag.add_documents(df)

/tmp/ipykernel_1851406/299196106.py:47: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(
/opt/conda/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dimension=1024


In [8]:
# test
query = "공에 힘이 주어졌을 때 공이 어떻게 움직이는지 과학적으로 설명해줘."
docs = science_rag.vectorstore.similarity_search_with_score(query, k=3)
docs

[(Document(id='e2161953-e80b-41b3-a83b-5bc7d76a801c', metadata={'docid': 'e2161953-e80b-41b3-a83b-5bc7d76a801c', 'content': '공이 땅에서 굴러가고 있습니다. 이때, 공이 움직이는 방향과 같은 방향으로 힘이 공을 밀어줍니다. 이렇게 힘이 가해지면 공은 원래 움직이던 방향으로 더 빠르게 움직입니다. 이는 힘이 공에 가해져서 공의 운동에 영향을 주기 때문입니다. 힘은 질량에 가해지는 힘과 방향에 따라 운동에 영향을 줄 수 있습니다. 따라서, 공이 힘을 받으면 힘이 가해진 방향과 같은 방향으로 더 빠르게 움직이게 됩니다.'}, page_content='공이 땅에서 굴러가고 있습니다. 이때, 공이 움직이는 방향과 같은 방향으로 힘이 공을 밀어줍니다. 이렇게 힘이 가해지면 공은 원래 움직이던 방향으로 더 빠르게 움직입니다. 이는 힘이 공에 가해져서 공의 운동에 영향을 주기 때문입니다. 힘은 질량에 가해지는 힘과 방향에 따라 운동에 영향을 줄 수 있습니다. 따라서, 공이 힘을 받으면 힘이 가해진 방향과 같은 방향으로 더 빠르게 움직이게 됩니다.'),
  np.float32(0.53543067)),
 (Document(id='a662feb5-e069-44d8-b148-4641039d2329', metadata={'docid': 'a662feb5-e069-44d8-b148-4641039d2329', 'content': '공을 공중에 직선으로 던져서 다시 내려올 때 공의 속도를 가장 알맞게 설명하는 것은 다음과 같습니다. 공은 올라갈수록 점점 느려지며, 멈춘 후, 내려올 때 점점 빨라집니다. 이는 중력의 영향으로 인해 발생하는 현상입니다. 중력은 공을 아래로 끌어당기는 힘으로, 공이 올라갈 때는 중력의 힘이 공의 운동에 반대 방향으로 작용하여 속도를 감소시킵니다. 그리고 공이 멈춘 후, 중력의 힘은 공을 아래로 끌어당기기 시작하여 공의 속도를 증가시킵니다. 이러한 과정을 통해 공은

In [9]:
# 5. 질의문 생성 체인

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

llm = ChatOllama(model="alibayram/Qwen3-30B-A3B-Instruct-2507")

# 프롬프트 
convertFormat = """
당신은 문자열 포맷 마이그레이션 전문가 입니다. <message> 에서 content 내용만 문자열로 출력합니다. 만일 content가 여러개가 있으면 전체적인 문맥을 파악하여 질문을 만들어서 출력합니다.

<message>
{message}
</message>

[example]

    <message>{{"role": "user", "content": "피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은?"}}</message> 
    output:피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은? 

    <message>{{"role": "user", "content": "이란 콘트라 사건이 뭐야"}}, {{"role": "assistant", "content": "이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다."}}, {{"role": "user", "content": "이 사건이 미국 정치에 미친 영향은?"}}</message>
    output:1986년에 발생한 이란 콘트라 사건이 미국 정치에 미친 영향은? 


output:
[Your output here - NOTHING ELSE]
"""

# 프롬프트 객체 생성
convertFormat_prompt = PromptTemplate(
    input_variables=["message"],
    template=convertFormat
)

# 출력 파서 (문자열)
output_parser = StrOutputParser()

# LCEL 체인 구성
convertFormat_chain = (
    convertFormat_prompt 
    | llm 
    | output_parser
)

In [10]:
# 6. 과학상식 체크 체인

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama


# 프롬프트 
selectYn = """
당신은 세상의 모든 상식과 지식에 정통한 전문가 입니다. 만약 <message> 가 세상의 상식 또는 지식에 관련된 질문이라면  Y 아니라면 N으로 답해주세요. 너에 관하여 물어보는건 세상의 상식 또는 지식에 관련된 질문이 아니야!

<message>
{message}
</message>

당신은 반드시 Y 뜨는 N으로 답해야 합니다.
Answer:
"""

# 프롬프트 객체 생성
selectYn_prompt = PromptTemplate(
    input_variables=["message"],
    template=selectYn
)

# 출력 파서 (문자열)
output_parser = StrOutputParser()

# LCEL 체인 구성
selectYn_chain = (
    selectYn_prompt 
    | llm 
    | output_parser
)

In [11]:
# 7. 답변 생성 체인

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama


# 프롬프트 
answer = """
당신은 과학 상식 전문가 입니다. <message> 의 질문에 대해서 주어진 <reference> 정보를 활용하여 간결하게 답변을 생성합니다.

    - 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답합니다.
    - 한국어로 답변을 생성합니다..

<message>
{message}
</message>

<reference>
{reference}
</reference>

Answer:
"""

# 프롬프트 객체 생성
answer_prompt = PromptTemplate(
    input_variables=["message"],
    template=answer
)

# 출력 파서 (문자열)
output_parser = StrOutputParser()

# LCEL 체인 구성
answer_chain = (
    answer_prompt 
    | llm 
    | output_parser
)

In [12]:
# 4-1. 리랭커 모델 로드 (새로운 셀 추가)
from sentence_transformers import CrossEncoder

# 한국어 리랭킹에 성능이 좋은 모델을 로드합니다.
reranker_model = CrossEncoder('BAAI/bge-reranker-v2-m3', max_length=512)

In [13]:
# # 8. 데이터 검색 함수

# def query_db(message):

#     docs = science_rag.vectorstore.similarity_search_with_relevance_scores(message, k=3)

#     #filtered_docs = []
#     content = []
#     docid= []
#     reference = []
    
#     for doc, score in docs:
#         content.append(doc.metadata['content'])
#         docid.append(doc.metadata['docid'])
#         reference.append({"score": float(score), "content": doc.metadata['content']})
#     return content, docid, reference

# 8. 데이터 검색 함수 (리랭킹 적용)

def query_db(message):
    
    # 1. 1차 검색(Retrieval): 리랭킹을 위해 후보군을 더 많이 가져옵니다 (예: k=10)
    retrieved_docs = science_rag.vectorstore.similarity_search_with_relevance_scores(message, k=40)

    # 2. 리랭킹(Re-ranking)
    # 리랭커의 입력 형식에 맞게 [질문, 문서] 쌍을 생성합니다.
    rerank_input_pairs = [[message, doc.page_content] for doc, score in retrieved_docs]
    
    # 리랭커 모델로 새로운 점수를 계산합니다.
    rerank_scores = reranker_model.predict(rerank_input_pairs)

    # 3. 새로운 점수와 기존 문서를 결합하여 정렬
    reranked_results = []
    for i, (doc, score) in enumerate(retrieved_docs):
        reranked_results.append({
            'doc': doc,
            'original_score': score,
            'rerank_score': rerank_scores[i]
        })
    
    # 리랭킹 점수(rerank_score)를 기준으로 내림차순 정렬
    reranked_results.sort(key=lambda x: x['rerank_score'], reverse=True)

    # 4. 최종 결과 선택: 리랭킹 후 상위 3개 문서를 선택
    final_docs = reranked_results[:3]

    # 5. 최종 결과 포맷팅 (기존 코드와 동일한 출력 형식 유지)
    content = []
    docid = []
    reference = []
    
    for result in final_docs:
        doc = result['doc']
        # reference에는 리랭킹된 점수를 넣어주는 것이 더 정확합니다.
        rerank_score = result['rerank_score']
        
        content.append(doc.metadata['content'])
        docid.append(doc.metadata['docid'])
        reference.append({"score": float(rerank_score), "content": doc.metadata['content']})
        
    return content, docid, reference

In [ ]:
# test

message = convertFormat_chain.invoke({"message": '{"role": "user", "content": "python 공부중인데..."}, {"role": "assistant", "content": "네 꼭 필요한 언어라서 공부해 두면 좋습니다."}, {"role": "user", "content": "숫자 계산을 위한 operator 우선순위에 대해 알려줘."}'})
print(message)
result = selectYn_chain.invoke({"message": "니가 대답을 잘해줘서 너무 신나!"})
result

In [ ]:
# test
import json

with open("../korea202/data/eval.jsonl") as f:
   for line in f:

            j = json.loads(line)
            message = convertFormat_chain.invoke({"message": j["msg"]})
            print(f'{message}:{len(message)}') 

In [ ]:
# test

content, docid, reference = query_db("세제의 거품이 만들어지는 원리는?")
print(content)
print(docid)
print(reference)

In [14]:
# 9. 메인 로직

import json

# 답변 데이터 생성
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    message = convertFormat_chain.invoke({"message": messages})
    result = selectYn_chain.invoke({"message": message})

    context = {"message":message}

    if result == "Y":
        context["reference"], response["topk"], response["references"] = query_db(message)
    else:
        context["reference"] = ""
        response["topk"] = []
        response["references"] = []
    
    response["answer"] = answer_chain.invoke(context)

    return response


# 답변 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:

            #if idx == 10: break
            
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [16]:
# 10. 메인 

# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("../korea202/data/eval.jsonl", "./data/sample_submission.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Answer: 나무의 분류를 조사하기 위해서는 먼저 나무의 생물학적 특징을 관찰하는 것이 중요합니다. 성장 속도, 크기, 온도 범위 등 외형적 특징을 비교하고, 특히 잎과 꽃 등의 구조적 특징을 세밀히 분석해야 합니다. 또한 현대 생물학에서는 유전자나 단백질의 분자 수준에서의 분석을 통해 진화적 관계를 파악함으로써 더 정확한 분류를 할 수 있습니다. 따라서 나무의 분류 조사를 위해서는 구조적 특징과 분자 생물학적 분석을 병행하는 것이 효과적입니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 주어진 정보에서는 각 나라의 공교육 지출 현황에 대한 구체적인 수치나 비교 데이터가 포함되어 있지 않으며, 전세계 공공 교육 지출이 세계 GDP의 약 4%를 차지한다는 전반적인 정보만 제공됩니다. 따라서 각 나라별 공교육 지출 현황에 대한 구체적인 답변을 드릴 수 없습니다.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]


KeyboardInterrupt: 